In [1]:
!python --version

Python 3.10.9


### Environnement d'exécution

In [2]:
import matplotlib.cm     as cm
import matplotlib.pyplot as plt
import matplotlib.tri    as tri
import numpy             as np
import os
import scipy.stats       as scs

### Fonctions génériques de traitement d'image

In [3]:
def autocorrelation(img: np.ndarray) -> np.ndarray:
    img_ = np.fft.fft2(img)
    img_ = np.power(np.abs(img_), 2)
    img_ = np.fft.ifft2(img_)
    img_ = np.abs(np.fft.fftshift(img_)/np.nanmax(img_))
    return img_

In [4]:
def cinfinity(img: np.ndarray) -> float:
    cinf = np.power(img.mean(), 2) /np.power(img, 2).mean()
    return cinf

In [5]:
def frequences(img: np.ndarray, shift=False) -> np.ndarray:
    img_ = np.fft.fft2(img)
    if shift: img_ = np.fft.fftshift(img_)
    img_ = np.abs(img_)
    return img_

In [6]:
def spatial(img: np.ndarray, shift=False) -> np.ndarray:
    img_ = np.fft.ifft2(img)
    if shift: img_ = np.fft.fftshift(img_)
    img_ = np.abs(img_)
    return img_

In [7]:
def profile(img: np.ndarray, theta: float) -> tuple:

    while theta > 2*np.pi:
        theta -= 2*np.pi
    
    while theta < 2*np.pi:
        theta += 2*np.pi

    if theta > np.pi:
        theta -= np.pi

    ny, nx = img.shape
    
    j_ = np.linspace(0, nx//2-1, nx//2).astype(np.int32)
    i_ = np.round(j_ *np.tan(theta)).astype(np.int32)

    j_ = j_[np.abs(i_) < ny//2]
    i_ = i_[np.abs(i_) < ny//2]

    radius = np.sqrt( np.power(i_, 2) + np.power(j_, 2) )

    if theta < np.pi/2:
        values = img[ny//2-i_, nx//2+j_]
    else:
        values = img[ny//2-i_, nx//2-j_]

    return (radius, values)

In [8]:
def contour(img: np.ndarray, cinf: float) -> tuple:

    ny, nx = img.shape
    
    angles = np.linspace(0, 179, 180).astype(np.int32)
    radius = np.zeros_like(angles, dtype=np.float64)

    j_ = np.linspace(0, nx//2-1, nx//2).astype(np.int32)

    for k, angle in enumerate(angles):
        
        if angle == 90:
            radius[k] = radius[k-1]
            continue

        i_ = np.round(j_ *np.tan(angle*np.pi/180)).astype(np.int32)
        j_ = j_[np.abs(i_) < ny//2]
        i_ = i_[np.abs(i_) < ny//2]

        radius[k] = np.sqrt( i_[-1]**2 + j_[-1]**2 )
        
        if angle < 90:
            for (i, j) in zip(i_,j_):
                if img[ny//2-i,nx//2+j] > cinf:
                    continue
                else:
                    radius[k] = np.sqrt( i**2 + j**2 )
                    break
        
        else:
            for (i, j) in zip(i_,j_):
                if img[ny//2-i,nx//2-j] >= cinf:
                    continue
                else:
                    radius[k] = np.sqrt( i**2 + j**2 )
                    break

    return (angles, radius)

### Fonctions d'affichage génériques

### Implémentation d'une expérience

In [9]:
class Experience():

    def __init__(self, infile: str, outfolder: str = ""):
        """
        Constructeur.

        Params
        ------
        infile    : str
                    lien absolu vers le fichier .csv contenant les données de l'expérience.
        outfolder : str (optional)
                    lien absolu vers le dossier dans lequel doit être enregistré le résultat de l'analyse.

        Return
        ------
        """
        self.path = infile
        self.outfolder = outfolder
        self.__read()
        self.__clean()
        self.__filter()
        return
    


    def __read(self):
        """
        Lecture du fichier .csv et paramétrage des attributs représentant la taille de l'image.
        """
        self.data = np.genfromtxt( self.path, delimiter=",", dtype=np.float64 )
        self.ny, self.nx = self.data.shape
        return
    

    def __clean(self):
        """
        Remplissage des NaNs détectés après la lecture. On remplit automatiquement par la moyenne.
        """
        self.data = np.nan_to_num(self.data, nan=np.nanmean(self.data))
        return
    

    def __filter(self):
        """
        Transformation d'une partie du spectre de l'image.
        """
        self.img = frequences(self.data, shift=True)
        self.img[:,self.nx//2] = np.power(self.img[:,self.nx//2], 0.5)
        self.img = spatial(self.img, shift=False)
        return



    def analyse(self):
        
        self.describe(self.outfolder)
        
        nrows = 1
        ncols = 4
        fig = plt.figure(figsize=(nrows*40, ncols*(8+0.5)))

        axe = fig.add_subplot(nrows, ncols, 1)
        axe.imshow(self.data, origin="lower")
        axe.set_title("$E_{yy}$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")


        freqs = frequences(self.data)

        axe = fig.add_subplot(nrows, ncols, 2)
        axe.imshow(np.log(freqs), origin="lower")
        axe.set_title("$\log(\mathcal{F}(E_{yy}))$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")


        axe = fig.add_subplot(nrows, ncols, 3)
        axe.imshow(np.log(self.img), origin="lower")
        axe.set_title("$\log(\mathcal{T}(E_{yy}))$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")


        autocor = autocorrelation(self.img)
        cinf    = cinfinity(self.data)

        axe = fig.add_subplot(nrows, ncols, 4)
        # axe.imshow(np.log(autocor/cinf), origin='lower')
        # axe.contour(np.log(autocor/cinf), [0.0, (autocor/cinf).max()], cmap=cm.jet)
        axe.imshow(autocor, origin='lower')
        axe.contour(autocor, [cinf, autocor.max()], cmap=cm.jet)
        # axe.set_title("$\log\left(\\frac{\mathcal{A}(E_{yy})}{C_\infty}\\right)$")
        axe.set_title("$\mathcal{A}(E_{yy})$")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")

        plt.suptitle(f"{os.path.basename(self.path)}", fontsize=40)

        if self.outfolder == "":
            plt.show()
        else:
            plt.savefig(f"{self.outfolder}/{os.path.basename(self.path)}_autocorrelation.jpg")
        
        plt.close()


        fig = plt.figure()

        angles, radius = contour(autocor, cinf)

        axe = fig.add_subplot(1,1,1)
        axe.plot(angles, radius)
        axe.set_title("Distance au contour en fonction de la direction")
        axe.set_xlabel("Angle [°]")
        axe.set_xlim(left=0, right=180)
        axe.set_ylabel("Radius to $C_\infty$ [px]")
        axe.set_ylim(bottom=0, top=radius.max())

        if self.outfolder == "":
            plt.show()
        else:
            plt.savefig(f"{self.outfolder}/{os.path.basename(self.path)}_radius_vs_angle.jpg")
        
        plt.close()

        return


    def describe(self):
        if self.outfolder == "":
            print(f"Data description :")
            print(f"------------------")
            print(f"dimension : {self.data.ndim}")
            print(f"shape     : {self.data.shape}")
            print(f"NaNs      : {np.any(np.isnan(self.data))}")
            print(f"mean      : {np.nanmean(self.data)}")
            print(f"std       : {np.nanstd(self.data)}")
            print(f"min       : {np.nanmin(self.data)}")
            print(f"max       : {np.nanmax(self.data)}")
            print(f"kurtosis  : {scs.kurtosis(self.data.flatten(), fisher=True, nan_policy='omit')}")
            print(f"skewness  : {scs.skew(self.data.flatten(), nan_policy='omit')}")
            print(f"C_inf     : {cinfinity(self.data)}")
            print()
            print(f"Image description :")
            print(f"-------------------")
            print(f"dimension : {self.img.ndim}")
            print(f"shape     : {self.img.shape}")
            print(f"NaNs      : {np.any(np.isnan(self.img))}")
            print(f"mean      : {np.nanmean(self.img)}")
            print(f"std       : {np.nanstd(self.img)}")
            print(f"min       : {np.nanmin(self.img)}")
            print(f"max       : {np.nanmax(self.img)}")
            print(f"kurtosis  : {scs.kurtosis(self.img.flatten(), fisher=True, nan_policy='omit')}")
            print(f"skewness  : {scs.skew(self.img.flatten(), nan_policy='omit')}")
            print(f"C_inf     : {cinfinity(self.img)}")

        else:
            with open(os.path.join(self.outfolder, f"{os.path.basename(self.path)}.txt"), "w") as file:
                file.write(f"Data description :\n")
                file.write(f"------------------\n")
                file.write(f"dimension : {self.data.ndim}\n")
                file.write(f"shape     : {self.data.shape}\n")
                file.write(f"NaNs      : {np.any(np.isnan(self.data))}\n")
                file.write(f"mean      : {np.nanmean(self.data)}\n")
                file.write(f"std       : {np.nanstd(self.data)}\n")
                file.write(f"min       : {np.nanmin(self.data)}\n")
                file.write(f"max       : {np.nanmax(self.data)}\n")
                file.write(f"kurtosis  : {scs.kurtosis(self.data.flatten(), fisher=True, nan_policy='omit')}\n")
                file.write(f"skewness  : {scs.skew(self.data.flatten(), nan_policy='omit')}\n")
                file.write(f"C_inf     : {cinfinity(self.data)}\n")
                file.write("\n")
                file.write(f"Image description :\n")
                file.write(f"-------------------\n")
                file.write(f"dimension : {self.img.ndim}\n")
                file.write(f"shape     : {self.img.shape}\n")
                file.write(f"NaNs      : {np.any(np.isnan(self.img))}\n")
                file.write(f"mean      : {np.nanmean(self.img)}\n")
                file.write(f"std       : {np.nanstd(self.img)}\n")
                file.write(f"min       : {np.nanmin(self.img)}\n")
                file.write(f"max       : {np.nanmax(self.img)}\n")
                file.write(f"kurtosis  : {scs.kurtosis(self.img.flatten(), fisher=True, nan_policy='omit')}\n")
                file.write(f"skewness  : {scs.skew(self.img.flatten(), nan_policy='omit')}\n")
                file.write(f"C_inf     : {cinfinity(self.img)}\n")
        return


    def plot_data(self, path=""):
        fig = plt.figure()
        
        axe = fig.add_subplot(1,1,1)
        axe.axis("equal")
        axe.imshow(self.data, origin="lower")
        axe.set_title(f"{os.path.basename(self.path)} - données brutes")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")
        
        if path == "":
            plt.show()
        else:
            plt.savefig(f"{path}/donnees.jpg")

        plt.close()
        return
    

    def plot_img(self):
        """
        Produit l'image au format .jpg de l'image obtenue après pré-traitement des données.
        
        Params
        ------
        
        Return
        ------
        """
        fig = plt.figure()
        
        axe = fig.add_subplot(1,1,1)
        axe.axis("equal")
        axe.imshow(np.log(self.img), origin="lower")
        axe.set_title(f"{os.path.basename(self.path)} - données pré-traitées ($\log$)")
        axe.set_xlabel("X [px]")
        axe.set_ylabel("Y [px]")
        
        if self.outfolder== "":
            plt.show()
        else:
            plt.savefig(f"{self.outfolder}/image.jpg")

        plt.close()
        return

### Implémentation d'une simulation numérique effectuée avec ADELI

Le post-traitement d'une expérience se fait à partir de la lecture des deux fichiers `t` et `p`, le premier contient le nombre de pas de temps, qui est un paramètre d'entrée de la lecture du fichier `p`.

In [10]:
class Tfile():

    def __init__(self, path: str) -> None:
        self.path = path
        return


    def read(self) -> np.ndarray:
        dates = None

        try:
            dates = np.genfromtxt(self.path)
            dates = dates[:, 1]

        except FileNotFoundError:
            print(f"{self.path} is not readable, skip.")

        return dates

Ainsi la classe `Tfile` ne contient qu'un chemin et a pour seule méthode l'extraction des valeurs des dates de sortie, dimensionnées. Le nombre d'éléments du vecteur renvoyé par la méthode `Tfile.read()` étant le nombre de dates. 

In [11]:
class Pfile():

    def __init__(self, path: str) -> None:
        self.path = path
        self.__read_header()
        
        return None
    

    def __read_header(self) -> None:

        fields   = []
        topology = []

        try:
            with open(self.path, "r") as stream:
                
                # Read fields related informations
                number = int(stream.readline().strip())
                for _ in range(number):
                    fields.append(stream.readline().strip())

                # Read topology related informations
                next(stream)
                topology = np.int_(list(filter(None, stream.readline().strip().split(" "))))

                # Update object attributes
                self.nfields = len(fields)
                self.fields  = fields

                self.nelem = topology[0]
                self.nvert = topology[1]
                self.ngaus = topology[2]
                self.ndime = topology[3]
                self.nface = topology[4]
                self.npres = topology[5]

                self.offset = 1 +self.nfields +3 +self.nelem +1 +(self.npres//10 +1) +1 +self.nface +2

        except FileNotFoundError:
            print(f"{self.path} is not readable, skip.")
        
        return None
    

    def read_elements(self) -> np.ndarray:
        """
        Lit la table de connectivité des noeuds du maillage.

        Params
        ------

        Return
        ------
        elements : np.ndarray
                   conteneur de taille (nombre d'elements x 5), respectivement index du materiau, puis quatre indices de noeuds.
        """
        elements = None

        try:
            elements = np.loadtxt( self.path,
                                   dtype=int,
                                   skiprows=(4 +self.nfields),
                                   max_rows=self.nelem )
            elements = elements[:, 1:]
        
        except FileNotFoundError:
            print(f"{self.path} is not readable, skip.")

        return elements
    

    def read_contour(self) -> np.ndarray:
        # contour = None

        # try:
        #     contour = np.genfromtxt( self.path,
        #                               dtype=int,
        #                               skip_header=(4 +self.nfields +self.nelem +1),
        #                               skip_footer=(self.npres//10 +1) )
        #     contour = contour.flatten()

        # except FileNotFoundError:
        #     print(f"{self.path} is not readable, skip.")

        # return contour
        raise NotImplementedError


    def read_faces(self) -> np.ndarray:
        """
        Lit les 
        """
        faces = None

        try:
            faces = np.loadtxt( self.path,
                                dtype=int,
                                skiprows=(4 +self.nfields +self.nelem +1 +int(self.npres//10 +1) +1),
                                max_rows=self.nface )
            faces = faces[:, 1:]

        except FileNotFoundError:
            print(f"{self.path} is not readable, skip.")

        return faces
    

    def read_coords(self, dates: list, names: list) -> np.ndarray:
        """
        Lit les coordonnées, i.e les grandeurs nodales, de la simulation.

        Params
        ------
        dates : list[int]
                liste d'entiers naturels correspondant aux index des dates.
        names : list[str]
                liste de noms de coordonnées tels que définis dans le script fortran extrac3d_ascii_plus_peierls.f.

        Return
        ------
        coords : np.ndarray
                 valeurs des coordonnées lues ; coords[i, j, k] est la coordonnée names[j] au noeud k au temps dates[i].
        """
        coords = np.zeros((len(dates), len(names), self.nvert)) *float('nan')

        coordsmap = { 'x' : 1, 'y' : 2, 'z' : 3,
                      'vx': 4, 'vy': 5, 'vz': 6,
                      'ux': 7, 'uy': 8, 'uz': 9,
                      'T' : 10 }
        
        try:
            for i, n in enumerate(dates):
                coords[i, :, :] = np.loadtxt( self.path, 
                                              dtype=float,
                                              skiprows=(self.offset +n*(self.nvert + self.nelem +3)),
                                              max_rows=self.nvert,
                                              usecols=[coordsmap[name] for name in names] ).transpose()
        
        except FileNotFoundError:
            print(f"{self.path} is not readable, skip.")

        return coords
    

    def read_fields(self, dates: list, names: list) -> np.ndarray:
        """
        Lit depuis le fichier de sortie d'ADELI les grandeurs élémentaires.

        Params
        ------
        dates : list[int]
                liste d'entiers naturels correspondant aux index des dates.
        names : list[str]
                liste des noms des champs tels que définis dans le script fortran 77 extrac3d_ascii_plus_peierls.f

        Return
        ------
        fields : np.ndarray
                 fields[i, j, k] est la valeur du champs names[j] évalué sur l'élément k au temps dates[i]
        """
        fields = np.zeros((len(dates), len(names), self.nelem)) *float('nan')

        fieldsmap = dict(
            zip(
                    [name for name in self.fields],
                    [self.fields.index(name)+1 for name in self.fields] 
               )
        )

        try:
            for i, n in enumerate(dates):
                fields[i, :, :] = np.loadtxt( self.path,
                                              dtype=float,
                                              skiprows=(self.offset +n*(self.nelem +3) +(n+1)*self.nvert),
                                              max_rows=self.nelem,
                                              usecols=[fieldsmap[name] for name in names] ).transpose()
                
        except FileNotFoundError:
            print(f"{self.path} is not readable, skip.")

        return fields

La classe `Pfile` représente le fichier `p` d'une simulation par son chemin d'accès. Ses méthodes sont multiples et renvoient à la diversité des informations qu'on peut trouver dans le fichier, en particulier on y lit :

- `Pfile.__read_header()` est une méthode privée qui permet l'accès aux informations nécessaires à la bonne lecture du fichier.

Les méthodes publiques sont :

- `Pfile.read_elements()`
- `Pfile.read_contour()`
- `Pfile.read_faces()`

Ces trois fonctions sont les fonctions de base permettant de reconstruire la géométrie du domaine étudié. En fait, elles représentent la *topologie* du maillage permettant de reconstruire, à chaque pas de temps, la géométrie du domaine.

- `Pfile.read_coords()`

permet d'accéder aux valeurs nodales des objets de la simulation. Les coordonnées des points du maillage en font partie, ainsi que la vitesse, le déplacement et la température.

- `Pfile.read_fields()`

renvoie quant à elle les valeurs des champs physiques ou de leurs coordonnées. Ce sont des champs tensoriels.

Les deux classes `ScalarField` et `TensorField` viennent représenter les deux cas.

In [29]:
class ScalarField():

    def __init__(self, name: str, dates: np.ndarray, mesh: list, values: np.ndarray) -> None:
        """
        Constructeur.

        Params
        ------
        name   : str
                 Nom du champ, utilisé dans les descriptions.
        dates  : np.ndarray
                 Vecteur des dates dimensionnées.
        mesh   : list
                 Liste des maillages de type tri.Triangulation.
        values : np.ndarray
                 values[k, i] représente la valeur du champ au k-ème instant, sur l'élément dont l'indexe est i.
        """
        self.name   = name
        self.dates  = dates
        self.idates = list(range(len(dates)))
        self.mesh   = mesh
        self.values = values
        
        return None
    


    def describe(self, outfolder: str = "") -> None:

        description = \
f"""Informations :
--------------
name  : {self.name}
shape : {self.values.shape}
NaNs  : {np.any(np.isnan(self.values))}
"""
        for k, d in enumerate(self.dates):
            description += "\n"
            description += \
f"""Temps {d} - {self.dates[k]} [an]
--------
min      : {np.nanmin(self.values[k])}
max      : {np.nanmax(self.values[k])}
mean     : {np.nanmean(self.values[k])}
std      : {np.nanstd(self.values[k])}
skew     : {scs.skew(self.values[k], nan_policy='omit')}
kurtosis : {scs.kurtosis(self.values[k], fisher=True, nan_policy='omit')}
"""

        if outfolder == "":
            print(description)
        else:
            with open(f"{outfolder}/{self.name}_description.txt", "w") as buffer:
                buffer.write(description)
        
        return None
    


    def interpolate(self) -> None:
        """
        Construit les attributs `grid` et `pict` de l'objet.
        """
        nt, nel = self.values.shape

        for n in nt:
            

On peut maintenant implémenter la classe `Simulation`, toute l'information liée à une simulation se trouve en principe dans un unique dossier, on peut donc la représenter par le chemin de ce dossier. Dans le cadre de ce projet, il ne s'agit que de post-traiter une simulation ADELI qui a déjà abouti, on ne s'intéresse donc qu'aux deux fichiers `t` et `p`. On peut facilement travailler sur cette classe pour y inclure les autres fichiers afin de paramétrer voire de gérer complètement les simulations numériques.

In [ ]:
class Simulation():

    def __init__(self, inpath: str, scalar=['Peierls', 'Work', 'WorkRate'], tensor=['d', 'e', 's']):
        """
        Constructeur.
        """
        self.path  = inpath
        self.pfile = Pfile(os.path.join(self.path, "".join([filename for filename in os.listdir(self.path) if filename.startswith("p")])))
        self.tfile = Tfile(os.path.join(self.path, "".join([filename for filename in os.listdir(self.path) if filename.startswith("t")])))
        
        self.dates  = self.tfile.read()
        self.scalar = scalar
        self.tensor = tensor
        return